In [2]:
import numpy
import pandas as pd
import numpy as np
import networkx as nx
from math import ceil
import os

In [3]:
def get_time_points(match_num):
    fullevent_data = pd.read_csv("./2020_Problem_D_DATA/fullevents.csv")
    pfed = fullevent_data.loc[:, ['MatchID', 'TeamID', 'MatchPeriod', 'EventTime', 'EventType', 'EventSubType']]
    pfed = pfed[pfed['MatchID']==match_num]
    pfed = pfed.reset_index()
    pfed = pfed.loc[:, ['MatchID', 'TeamID', 'MatchPeriod', 'EventTime', 'EventType', 'EventSubType']]
    last_1H = max(pfed[pfed['MatchPeriod']=='1H'].EventTime)

    fed_2H = pfed[pfed['MatchPeriod']=='2H']
    fed_2H['EventTime'] += last_1H

    pfed = pd.concat([pfed[pfed['MatchPeriod']=='1H'], fed_2H])

    points = [0]

    points.append(min(pfed[pfed['MatchPeriod']=='2H'].index))

    sindex = list(pfed[pfed['EventType']=='Substitution'].index)

    for p in sindex:
        points.append(p)

    points.append(max(pfed.index))

    points = list(set(points))
    points = sorted(points)

    time_points = []

    for i in range(len(points)-1):
        start_time = pfed.iloc[points[i]].EventTime
        end_time = pfed.iloc[points[i+1]].EventTime
        # print(start_time, end_time)
        # time_points.append(start_time)

        duration = end_time - start_time
        num_points = ceil(duration / 360)
        if num_points <= 0:
            continue
        sub_quantum = duration / num_points

        while start_time < end_time:
            time_points.append(start_time)
            start_time += sub_quantum

    time_points.append(pfed.iloc[points[-1]].EventTime)

    return time_points

In [4]:
def get_raw_data_filtered(match_num, is_oppnent):
    if is_oppnent:
        player_list = list(set(fullevent_data.TeamID) - {'Huskies'})
    else:
        player_list = ['Huskies']
    
    passing_data = pd.read_csv("./2020_Problem_D_DATA/passingevents.csv")
    PPD = passing_data.loc[:, ['MatchID', 'TeamID', 'OriginPlayerID', 'DestinationPlayerID', 'EventTime', 'EventOrigin_x', 'EventOrigin_y', 'EventDestination_x', 'EventDestination_y']]
    PPD = PPD[PPD['MatchID']==match_num][PPD['TeamID'].isin(player_list)]

    return PPD

In [17]:
for match in range(1, 39):

    print("No. {}".format(match))

    base_path = './results/short/match_{}/'.format(match)
    if not os.path.exists(base_path):
        os.makedirs(base_path)

    ecds = []
    acs = []
    notc = []
    lpln = [] # long passing link number

    PPD = get_raw_data_filtered(match, False)
    time_points = get_time_points(match)

    for i in range(len(time_points)-1):
        start_time = time_points[i]
        end_time = time_points[i+1]
        if i == len(time_points)-2:
            end_time += 360

        ppd = PPD[PPD.EventTime >= start_time][PPD.EventTime < end_time]

        # ecds.append(get_ecd(ppd.copy()))
        # acs.append(get_ac(ppd.copy()))
        # get_clustering(ppd.copy(), i)
        # notc.append(get_tc(ppd.copy()))
        # get_tf(ppd.copy(), i)
        lpln.append(get_long_link_num(ppd.copy()))

    # save as csv
    # pd.DataFrame({'ECD':ecds}).to_csv(base_path + 'ECD.csv')
    # pd.DataFrame({'AC':acs}).to_csv(base_path + 'AC.csv')
    # pd.DataFrame({'num of TC':notc}).to_csv(base_path + 'TC.csv')
    pd.DataFrame({'num of long passing link':lpln}).to_csv(base_path + 'LongPassingLink.csv')

No. 1
No. 2
No. 3
No. 4
No. 5
No. 6
No. 7
No. 8
No. 9
No. 10
No. 11
No. 12
No. 13
No. 14
No. 15
No. 16
No. 17
No. 18
No. 19
No. 20
No. 21
No. 22
No. 23
No. 24
No. 25
No. 26
No. 27
No. 28
No. 29
No. 30
No. 31
No. 32
No. 33
No. 34
No. 35
No. 36
No. 37
No. 38


In [5]:
def get_ecd(ppd):

    # ppd = PPD[PPD.EventTime >= start_time][PPD.EventTime < end_time]
    ppd 
    ppd = ppd.rename(columns={'OriginPlayerID':'i', 'DestinationPlayerID':'j'})
    ppd['count'] = 1
    ppd = ppd.groupby(['i', 'j']).sum()
    ppd = ppd.sort_values(by='count').ix[::-1]

    ppd = pd.DataFrame({'i': list(map(lambda x: x[0], ppd.index)), 'j': list(map(lambda x: x[1], ppd.index)), 'count':ppd.to_numpy().T[0]})
    
    players = list(set(ppd['i']))
    players = sorted(players)

    num_players = len(players)

    mat = np.zeros((num_players, num_players))

    for i, iplayer in enumerate(players):
        for j, jplayer in enumerate(players):
            qr = ppd[ppd['i']==iplayer][ppd['j']==jplayer].to_numpy()
            if len(qr) > 0:
                mat[i][j] = qr[-1][-1]
            else:
                mat[i][j] = 0
    
    A = mat

    G = nx.Graph()

    for i, p in enumerate(players):
        G.add_node(i)

    for i in range(len(players)):
        for j in range(len(players)):
            G.add_edge(i, j, weight=A[i][j])
    try:
        ec = nx.eigenvector_centrality_numpy(G, weight='weight')
    except:
        # print(players)
        return None

    ps = []
    vs = []
    for p, v in ec.items():
        ps.append(p)
        vs.append(v)

    max_ec = max(vs)
    player_with_ec = ps[vs.index(max_ec)]

    dispersion_ec = np.std(vs)

    return dispersion_ec

# pd.DataFrame({'dispersion':dispersions}).to_csv('./results/short/match_{}/Eigenvector centrality dispersion.csv'.format(match))

In [6]:
def get_ac(ppd):

    # ppd = PPD[PPD.EventTime >= start_time][PPD.EventTime < end_time]
    ppd = ppd.rename(columns={'OriginPlayerID':'i', 'DestinationPlayerID':'j'})
    ppd['count'] = 1
    ppd = ppd.groupby(['i', 'j']).sum()
    ppd = ppd.sort_values(by='count').ix[::-1]

    ppd = pd.DataFrame({'i': list(map(lambda x: x[0], ppd.index)), 'j': list(map(lambda x: x[1], ppd.index)), 'count':ppd.to_numpy().T[0]})
    
    players = list(set(ppd['i']))
    players = sorted(players)

    num_players = len(players)

    mat = np.zeros((num_players, num_players))

    for i, iplayer in enumerate(players):
        for j, jplayer in enumerate(players):
            qr = ppd[ppd['i']==iplayer][ppd['j']==jplayer].to_numpy()
            if len(qr) > 0:
                mat[i][j] = qr[-1][-1]
            else:
                mat[i][j] = 0
    
    A = mat

    S = np.zeros((num_players, num_players))

    for i, line in enumerate(A):
        S[i][i] = sum(line)

    L = S - A

    eigenvalueL, eigenvectorL = np.linalg.eig(L)

    if len(eigenvalueL) == 0:
        return None
    elif len(eigenvalueL) == 1:
        return eigenvalueL[0]

    return sorted(eigenvalueL)[1]

# pd.DataFrame({'dispersion':dispersions}).to_csv('./results/short/match_{}/Eigenvector centrality dispersion.csv'.format(match))

In [7]:
def get_clustering(ppd, period):

    # ppd = PPD[PPD.EventTime >= start_time][PPD.EventTime < end_time]
    ppd = ppd.rename(columns={'OriginPlayerID':'i', 'DestinationPlayerID':'j'})
    ppd['count'] = 1
    ppd = ppd.groupby(['i', 'j']).sum()
    ppd = ppd.sort_values(by='count').ix[::-1]

    ppd = pd.DataFrame({'i': list(map(lambda x: x[0], ppd.index)), 'j': list(map(lambda x: x[1], ppd.index)), 'count':ppd.to_numpy().T[0]})
    
    players = list(set(ppd['i']))
    players = sorted(players)

    num_players = len(players)

    d0 = ppd
    d1 = d0.copy()
    d2 = d0.copy()

    d1 = d1.rename(columns={'i':'j', "j":'k'})
    d2 = d2.rename(columns={'i':'k', "j":'i'})

    merged_data = pd.merge(pd.merge(d0, d1, on='j'), d2, on=['k', 'i'])
    md = merged_data
    md = md.rename(columns={'count_x':'wij', 'count_y':'wjk', 'count':'wki'})

    md['www'] = [(wij * wjk * wki) for wij, wjk, wki in zip(md['wij'], md['wjk'], md['wki'])]
    md['ww'] = [(wij * wki) for wij, wki in zip(md['wij'], md['wki'])]

    md = md.groupby('i').sum()

    md['Clustering coefficient'] = [www/ww for www,ww in zip(md['www'], md['ww'])]
    md = md.sort_values(by='Clustering coefficient').ix[::-1]

    md.to_csv(base_path + 'period-{}-clustering.csv'.format(period))

# pd.DataFrame({'dispersion':dispersions}).to_csv('./results/short/match_{}/Eigenvector centrality dispersion.csv'.format(match))

In [8]:
from itertools import combinations, permutations

def get_tc(ppd):

    # ppd = PPD[PPD.EventTime >= start_time][PPD.EventTime < end_time]
    ppd = ppd.rename(columns={'OriginPlayerID':'i', 'DestinationPlayerID':'j'})
    ppd['count'] = 1
    ppd = ppd.groupby(['i', 'j']).sum()
    ppd = ppd.sort_values(by='count').ix[::-1]

    ppd = pd.DataFrame({'i': list(map(lambda x: x[0], ppd.index)), 'j': list(map(lambda x: x[1], ppd.index)), 'count':ppd.to_numpy().T[0]})
    
    players = list(set(ppd['i']))
    players = sorted(players)

    num_players = len(players)

    triples = combinations(players, 3)
    triples = list(triples)

    all_edges = [(x, y) for x,y in zip(ppd.i, ppd.j)]

    triangles = []

    for a, b, c in triples:
        if ((a,b) in all_edges or (b,a) in all_edges) and ((a,c) in all_edges or (c,a) in all_edges) and ((b,c) in all_edges or (c,b) in all_edges):
            triangles.append((a,b,c))

    return len(triangles)

# pd.DataFrame({'dispersion':dispersions}).to_csv('./results/short/match_{}/Eigenvector centrality dispersion.csv'.format(match))

In [9]:
from itertools import combinations, permutations

def get_tf(ppd, period):

    # ppd = PPD[PPD.EventTime >= start_time][PPD.EventTime < end_time]
    ppd = ppd.rename(columns={'OriginPlayerID':'i', 'DestinationPlayerID':'j'})
    ppd['count'] = 1
    ppd = ppd.groupby(['i', 'j']).sum()
    ppd = ppd.sort_values(by='count').ix[::-1]

    ppd = pd.DataFrame({'i': list(map(lambda x: x[0], ppd.index)), 'j': list(map(lambda x: x[1], ppd.index)), 'count':ppd.to_numpy().T[0]})
    
    players = list(set(ppd['i']))
    players = sorted(players)

    num_players = len(players)

    triples = combinations(players, 3)
    triples = list(triples)

    all_edges = [(x, y) for x,y in zip(ppd.i, ppd.j)]

    triangles = []

    for a, b, c in triples:
        if ((a,b) in all_edges or (b,a) in all_edges) and ((a,c) in all_edges or (c,a) in all_edges) and ((b,c) in all_edges or (c,b) in all_edges):
            triangles.append((a,b,c))

    values = [0] * len(triangles)

    for i, t in enumerate(triangles):
        edges = list(permutations(t, 2))
        for e in edges:
            qr = ppd[ppd.i == e[0]][ppd.j == e[1]].to_numpy()
            if len(qr) > 0:
                values[i] += qr[-1][-1]

    tvd = pd.DataFrame({'triangle':triangles, 'value':values})
    tvd = tvd.sort_values(by='value').ix[::-1]

    if len(tvd.value) <= 0:
        return None

    max_value = max(tvd.value)
    
    threshold = max_value * 0.75

    triangles_filtered = tvd[tvd['value'] >= threshold]
    tfd = triangles_filtered

    def count_shared_node(t0, t1):
        return len(set(t0) & set(t1)) 

    tf = list(tfd.triangle.to_numpy())
    pairs = combinations(tf, 2)
    pairs = list(pairs)

    weight = [count_shared_node(t0, t1)*(tvd[tvd.triangle==t0].value.to_numpy()[0]+tvd[tvd.triangle==t1].value.to_numpy()[0])/2 for t0, t1 in pairs]

    a = [p[0] for p in pairs]
    b = [p[1] for p in pairs]

    new_graph = pd.DataFrame({'i':a, 'j':b, 'weight':weight})

    new_graph = new_graph[new_graph['weight'] > 0]

    newMat = np.zeros((len(tf), len(tf)))
    for i, inode in enumerate(tf):
        for j, jnode in enumerate(tf):
            if inode==jnode:
                newMat[i][j] = 3 * tvd[tvd.triangle==inode].value.to_numpy()[0]
                continue
            qr = new_graph[new_graph['i']==inode][new_graph['j']==jnode].to_numpy()
            if len(qr) > 0:
                newMat[i][j] = qr[-1][-1]

    new_eigenvalue, new_eigenvector = np.linalg.eig(newMat)
    max_new_eigenvalue = max(new_eigenvalue)

    new_graph['eigenvalue'] = max_new_eigenvalue

    new_graph.to_csv(base_path + 'period-{}-team_inf.csv'.format(period))

In [15]:
def get_long_link_num(ppd):

    # ppd = PPD[PPD.EventTime >= start_time][PPD.EventTime < end_time]
    ppd = ppd.rename(columns={'OriginPlayerID':'i', 'DestinationPlayerID':'j'})

    pairs = [(i, j) for i,j in zip(ppd.i, ppd.j)]

    total = 0
    count = 0
    for j in range(1, len(pairs)):
        if pairs[j][0] == pairs[j-1][1]:
            count += 1
        else:
            if count >= 2:
                total += 1
            count = 0
    
    return total

In [30]:
ppd['count'] = 1
ppd = ppd.groupby(['OriginPlayerID', 'DestinationPlayerID']).sum()
ppd = ppd.sort_values(by='count').ix[::-1]

# pd.DataFrame({'i': list(map(lambda x: x[0], ppd.index)), 'j': list(map(lambda x: x[1], ppd.index)), 'count':ppd.to_numpy().T[0]})
ppd

,,MatchID,EventTime,EventOrigin_x,EventOrigin_y,EventDestination_x,EventDestination_y,count
OriginPlayerID,DestinationPlayerID,,,,,,,
Huskies_D2,Huskies_D3,18,21718.423754,627,1010,723.0,347.0,18
Huskies_D1,Huskies_D2,14,17440.997597,441,1185,432.0,767.0,14
Huskies_F2,Huskies_M1,14,25153.278839,902,1034,886.0,1045.0,14
Huskies_M1,Huskies_D1,13,13900.565449,582,1084,480.0,1049.0,13
Huskies_D1,Huskies_M1,11,14692.174254,449,905,527.0,972.0,11
...,...,...,...,...,...,...,...,...
Huskies_D4,Huskies_F1,1,80.501137,47,3,65.0,27.0,1
Huskies_M2,Huskies_F1,1,1704.039030,23,76,47.0,96.0,1
Huskies_D4,Huskies_D2,1,1237.056443,19,5,21.0,39.0,1


In [14]:
lpln

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]